<a href="https://colab.research.google.com/github/gmsarti/historical-games/blob/main/jogos_historicos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic structure
We will need to install some libraries for this work.

I'm making this project in Google Colab and needed to use *openai<1.0.0* to make it work.

You will need to set your *openai API key* in this first.


In [ ]:
from IPython.display import clear_output

%pip install langchain
%pip install "openai<1.0.0"
!pip install duckduckgo_search
!pip install wikipedia


import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

clear_output()

# Simple RAG with Langchain

Here we use *ChatOpenAI* to retrieve information and later organize the information.

Langchain is used here to set a custom prompt and a parser.

The prompt used has been giving good results delivering a string in the format of a python dictionary. In any case I'll try to make in other way because I think I can retrieve better information with prompts specifically for the social context of the game and it's rules.

In [ ]:
# IMPORTs
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import(
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)
from langchain.schema import BaseOutputParser


# INSTANCIATE CHAT MODEL
chat_model = ChatOpenAI(model_name="gpt-4")
# chat_model3 = ChatOpenAI()


# MAKE CUSOM PROMPT
system_template = """
You are a helpful assistant who returns python dictionaies with information about games.
Given the name of the game return the following informations:
    country: list of strings with the countries were the game is from,
    social: string with history of the game,
    players:interger with the number of players for that game,
    pieces: string describing the pieces necessary for play the game,
    rules: rules of the game.

"""

human_template = "{game_name}"

system_message = SystemMessagePromptTemplate.from_template(
    system_template
)

print(system_message)

human_message = HumanMessagePromptTemplate.from_template(
    human_template
)
print(human_message)

prompt = ChatPromptTemplate.from_messages([
    system_message, human_message
])


# MAKE PARSER
class DictParser(BaseOutputParser):

    def parse(self, text):
        output = eval(text)
        return output


# INSTANCIATE AND RUN CHAIN
chain = LLMChain(
    llm = chat_model,
    prompt = prompt,
    output_parser=DictParser(),
    verbose = False
)

print('\n\n')
chain.run("Yoté")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


prompt=PromptTemplate(input_variables=[], template='\nYou are a helpful assistant who returns python dictionaies with information about games.\nGiven the name of the game return the following informations:\n    country: list of strings with the countries were the game is from,\n    social: string with history of the game,\n    players:interger with the number of players for that game,\n    pieces: string describing the pieces necessary for play the game,\n    rules: rules of the game.\n\n')
prompt=PromptTemplate(input_variables=['game_name'], template='{game_name}')





{'country': ['Senegal', 'Ivory Coast', 'Mali'],
 'social': 'Yoté is a strategic board game, popular in West Africa. The game has simple rules, but deep strategy with many variations.',
 'players': 2,
 'pieces': 'The game is played with a 5x6 grid board and 24 pieces, 12 for each player. The pieces are usually small round disks.',
 'rules': "Each player begins the game with 12 pieces located on the first two rows of their side of the board. The players take turns moving one of their pieces to an empty square: Moving to an orthogonally adjacent square (not diagonally); or jumping over a piece (the player's own or the opponent's), if there is an empty square on the other side. A player captures an opponent's piece by jumping over it. The goal of the game is to capture all the opponent's pieces."}

# Breaking the RAG process for optimization

First step is to create a basic structure for the project.

In [ ]:
# IMPORTs
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import(
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate
)
from langchain.schema import BaseOutputParser


# INSTANCIATE CHAT MODEL
chat_model = ChatOpenAI(model_name="gpt-4")
# chat_model3 = ChatOpenAI()

# MAKE PARSER
class GameParser(BaseOutputParser):

    def parse(self, text):
        output = eval(text)
        return output



def make_prompt(prompt_text: str):
    system_template = prompt_text

    human_template = "{game_name}"

    system_message = SystemMessagePromptTemplate.from_template(
        system_template
    )

    print(system_message)

    human_message = HumanMessagePromptTemplate.from_template(
        human_template
    )
    print(human_message)

    prompt = ChatPromptTemplate.from_messages([
        system_message, human_message
    ])

    return prompt


def rag(prompt_text: str, game_name: str, parser: bool=False):
    if parser:
        chain = LLMChain(
            llm = chat_model,
            prompt = make_prompt(prompt_text),
            output_parser=GameParser(),
            verbose = False
        )
    else:
        chain = LLMChain(
            llm = chat_model,
            prompt = make_prompt(prompt_text),
            verbose = False
        )

    print('\n\n')
    return chain.run(game_name)

Let's test it...

I'll use the same parameters from the first attempt.

In [ ]:
prompt_text = """
You are a helpful assistant who returns python dictionaies with information about games.
Given the name of the game return the following informations:
    country: list of strings with the countries were the game is from,
    social: string with history of the game,
    players:interger with the number of players for that game,
    pieces: string describing the pieces necessary for play the game,
    rules: rules of the game.

"""

rag(
    prompt_text=prompt_text,
    game_name='Yoté',
    parser=True
)

prompt=PromptTemplate(input_variables=[], template="\nYou are a helpful assistant who returns python dictionaies with information about games.\nGiven the name of the game return the following informations:\n    country: list of strings with the countries were the game is from,\n    social: string with history of the game,\n    players:interger with the number of players for that game,\n    pieces: string describing the pieces necessary for play the game,\n    rules: rules of the game.\n\n\n\n Here are some informations about the game so you can build on:\n\nYoté is a traditional strategy board game of West Africa, where it is a popular gambling game due to its fast pace and surprising turnarounds. A player wins by capturing all opposing pieces. Yoté is related to the game Choko.\n\n\n== Rules ==\nThe game is played on a 5×6 board, which is empty at the beginning of the game. Each player has twelve pieces in hand. Players alternate turns, with White moving first. In a move, a player may

{'country': ['West Africa'],
 'social': 'Yoté is a traditional strategy board game of West Africa, where it is a popular gambling game due to its fast pace and surprising turnarounds. A player wins by capturing all opposing pieces. Yoté is related to the game Choko.',
 'players': 2,
 'pieces': 'Each player has twelve pieces in hand.',
 'rules': "The game is played on a 5×6 board, which is empty at the beginning of the game. Each player has twelve pieces in hand. Players alternate turns, with White moving first. In a move, a player may either:\n\nPlace a piece in hand on any empty cell of the board.\nMove one of their pieces already on the board orthogonally to an empty adjacent cell.\nCapture an opponent's piece if it is orthogonally adjacent to a player's piece, by jumping to the empty cell immediately beyond it. The captured piece is removed from the board, and the capturing player removes another of the opponent's pieces of his choosing from the board.The player who captures all the

The test seems to bring good results. There is a error in the rules, but this text will be revised in the end.

Now I can use the rag function in a GameGenerator class to try to improve the results.

In [ ]:
from dataclasses import dataclass
from typing import Any, Optional, List
from duckduckgo_search import DDGS
from bs4 import BeautifulSoup
import requests


@dataclass
class GameGenerator:
    game_name: str
    social: Optional[str] = None
    rules: Optional[str] = None
    basics: Optional[dict] = None
    img_links: Optional[List] = None

    # def make(self, game_name: str):
    def make(self):
        # self.game_name = game_name
        self.basics = self.game_basics(self.game_name)
        self.social = self.set_social(self.game_name)
        self.rules = self.set_rules(self.game_name)
        self.img_links = self.search_duckduckgo_images(self.game_name)

        return self.basics, self.social, self.rules, self.img_links

    def game_basics(self, game_name: str):
        prompt_text = """
            You are a helpful assistant who returns python dictionaies with information about games.
            You will ONLY GIVE INFORMATION YOU KNOW AND WILL NEVER MAKE STUFF UP. If you don't have the information, please, return None.
            Given the name of the game return the following informations:
                country: list of strings with the countries were the game is from,
                date: string with the century when the game was created,
                players:interger with the number of players for that game,
                pieces: string describing the pieces necessary for play the game,

                If you don't know some of this informations, please, return unknown for that information.
                    """
        return rag(
            prompt_text=prompt_text,
            game_name=self.game_name,
            parser=True
        )

    def set_social(self, name: str):
        prompt_text = """
            You are a helpful assistant who returns python dictionaies with information about games.
            You will ONLY GIVE INFORMATION YOU KNOW AND WILL NEVER MAKE STUFF UP. If you don't have the information, please, return None.
            Given the name of the game return informations of social relevance about the game. Try to write 2 or 3 paragraphs about it.
            The informations may be one or more facts like:
                What culture used to play it?
                There is a date of creation of this game? If not since when it's known to be played?
                It's associated to some religion or festivity?
                People still play this game?
                There is any famous competition of it? Any famous player?
                This game is mentioned in any books? If it is mentioned what is the name of the book and who is it's author?
                    """
        return rag(
            prompt_text=prompt_text,
            game_name=self.game_name,
        )

    def set_rules(self, name: str):
        prompt_text = """
            You are a helpful assistant who returns python dictionaies with information about games.
            You will ONLY GIVE INFORMATION YOU KNOW AND WILL NEVER MAKE STUFF UP. If you don't have the information, please, return None.
            Given the name of the game return it's rules.
            Be very detailed about the rules informing as if you're explaining to a kid
                    """
        return rag(
            prompt_text=prompt_text,
            game_name=self.game_name,
        )

    def search_duckduckgo_images(self, name: str):
        d = DDGS()
        search_results = d.images(f'tabuleiro do jogo {name}')

        image_links = []
        for result in search_results:
            try:
                image_link = result.get('image')
                if image_link:
                    image_links.append(image_link)
            except Exception as e:
                print(f"Error fetching image link: {e}")

        return image_links[:6]

## DO NOT TRUST BLINDLY IN THIS INFORMATION!!!
The code works and this is awesome!

Chat-GPT gives a lot of wrong informations thought. Specially in the social description. There is important informations missing about the Yoté, but it's able to give us quite a good start.

In [ ]:
game = GameGenerator('Yoté')
game.make()

prompt=PromptTemplate(input_variables=[], template="\n            You are a helpful assistant who returns python dictionaies with information about games.\n            You will ONLY GIVE INFORMATION YOU KNOW AND WILL NEVER MAKE STUFF UP. If you don't have the information, please, return None.\n            Given the name of the game return the following informations:\n                country: list of strings with the countries were the game is from,\n                date: string with the century when the game was created,\n                players:interger with the number of players for that game,\n                pieces: string describing the pieces necessary for play the game,\n\n                If you don't know some of this informations, please, return unknown for that information.\n                    \n\n Here are some informations about the game so you can build on:\n\nYoté is a traditional strategy board game of West Africa, where it is a popular gambling game due to its fast pac

({'country': ['West Africa'],
  'date': 'unknown',
  'players': 2,
  'pieces': 'A 5x6 board and 24 pieces, 12 for each player'},
 'Yoté is a traditional strategy board game that originated in West Africa. It is highly popular due to its exciting pace and unpredictable turn of events. The game is often associated with gambling activities, given its competitive nature. Yoté has a relation to the game Choko, indicating a linkage in their game rules or play styles.\n\nThis game is played on a 5×6 board and each player begins the game with twelve pieces in hand. The objective of the game is to capture all of the opponent\'s pieces. The players take turns, starting with the white pieces. The game allows for different kinds of moves, including placing a piece on any empty cell of the board, moving a piece already on the board to an adjacent cell, or capturing an opponent\'s piece. There are also optional rules that allow for additional strategic layers, such as multiple successive jumps in a 

# Put the data in a .md file
The idea here is to make an book with information about several games.

For that we can store this information in a text file os a markdown file to indicate some of the layout patterns.

In [ ]:
import os
from pathlib import Path

def add_info_to_md(file_path: str, game_name:str, game_data: tuple):
    # Create a Path object for the file
    path = Path(file_path)

    # Unpack game data
    game_basics, social, rules, image_urls = game_data

    # Prepare the image lines
    image_lines = "\n".join([f"<img src='{url}' width='100' height='100'>" for url in image_urls])

    # Combine the title, text, and images into a single string
    content = f"# {game_name}\n{social}\n## Rules\n{rules}\n### Images for reference\n{image_lines}\n\n\n"

    # Check if the file exists
    if path.exists():
        # Open the file in append mode ('a')
        with open(path, 'a') as f:
            # Write the new information to the file
            f.write(f"\n\n{content}\n")
    else:
        # If the file doesn't exist, create it and write the new information
        with open(path, 'w') as f:
            f.write(content + "\n")


In [ ]:
game_name = 'passe trappe'

# game_gen = GameGenerator(game_name)
# game = game_gen.make()

add_info_to_md(
    file_path = 'enciclopedia.md',
    game_name = game_name,
    game_data = game
)

# A batch of games

There will be some games in this book. But which ones?

Now is time to select the games to add.

With them selected loop the code above and make your book.

There is a great list of games in this [List of abstract games](https://en.wikipedia.org/wiki/List_of_abstract_strategy_games).

In [ ]:
import json


games = ["Abalone", "Adugo", "Agon", "Alquerque", "Amazons",
         "Arimaa", "Armenian checkers", "Asalto", "Astar", "Ataxx",
         "Awithlaknakwe", "Awithlaknannai Mosona", "Bagh bandi", "Bagha-chall",
         "Bizingo", "Blokus", "Blue and Gray", "Brax (game)", "Breakthrough",
         "Breakthru", "Buga-shadara", "Butterfly", "Camelot", "Canadian checkers",
         "Catch the hare (aka cercar la liebre or corner the rabbit)",
         "Cathedral", "Chinese checkers", "Choko", "Chopsticks", "Cinc camins",
         "Conga or celim", "Congo", "Conspirateurs", "Conquest", "Crossings",
         "Crosstrack", "Dablot Prejjesne", "Dash-guti", "Death stacks",
         "Demala diviyan keliya", "Diaballik", "Diamond", "Dominoes",
         "Draughts (also known as checkers)", "Egara-guti", "En Gehé",
         "Entropy (1977)", "Entropy (1994)", "Epaminondas", "Fanorona",
         "Felli", "Fetaix", "Fitchneal", "Five-ield kono", "Four-field kono",
         "Fox games, such as fox and geese", "Game of the Generals",
         "Go", "Gol-skuish", "Halatafl", "Halma", "Hare games", "Hare & Tortoise",
         "Hat diviyan keliya", "Hexagony", "High jump", "Hive", "Hnefatafl",
         "Indian and jackrabbits", "Ingenious", "Irensei", "Isola",
         "Italian Damone", "Jarmo", "Jul-gonu", "Jungle (dou shou qi, the game of fighting animals)",
         "Kalah", "Kaooa", "Kensington", "Kharbaga", "Khet", "Kolowis Awithlaknannai (fighting serpents)",
         "Komikan", "Konane", "Kotu Ellima", "L game", "Lau kata kati",
         "Leap Frog (board game)", "Len Choa", "Liberian Queah", "Lines of Action",
         "Lotus", "Ludus latrunculorum", "Main tapal empat", "Mak-yek", "Makonn",
         "Mancala and related games", "Martian chess (for two to six players)",
         "Meurimueng-rimueng-do", "Meurimueng-rimueng peuet ploh or Dam-daman or Ratti-chitti-bakri",
         "Ming mang", "Mozaic", "Neutron", "Nim †", "Paddles", "Pasang",
         "Pentagame", "Peralikatuma", "Permainan-Tabal", "Phutball", "Pretwa",
         "Pulijudam", "Quoridor", "Reversi, also known as Othello", "Rhumb Line",
         "Rhythmomachy", "Rimau", "Rimau-rimau", "Ringo", "Salta", "Sher-bakar",
         "Sixteen soldiers (aka \"cows and leopards\" or \"Sholo guti\")", "Stratego",
         "Sua ghin gnua (aka tigers and oxen)", "Surakarta", "Sz'swa",
         "Tafl games", "Terhuchu", "Terrace", "Three musketeers", "Thud",
         "Tiger and buffaloes", "Tuknanavuhpi", "Tukvnanawopi", "Turkish draughts",
         "Ugolki", "Xoliba", "Yoté", "Zamma"
]

to_be_processed = games.copy()
done = []
error = []

# Convert the list to a JSON object
to_be_processed_json = json.dumps(to_be_processed)

In [ ]:
for game_name in games:

    try:
        game_gen = GameGenerator(game_name)
        game_data = game_gen.make()

        add_info_to_md(
            file_path = '/content/drive/MyDrive/Colab Notebooks/MaquinaTudo/enciclopedia.md',
            # file_path = 'enciclopedia.md',
            game_name = game_name,
            game_data = game_data
        )

        to_be_processed.remove(game_name)
        done.append(game_name)

    except:
        to_be_processed.remove(game_name)
        error.append(game_name)
    clear_output()
    print(f"{game_name}\n\nGames to do: {len(to_be_processed)}\nDone: {len(done)}\nError: {len(error)}\n\n\n")

Zamma

Games to do: 0
Done: 125
Error: 14



